In [1]:
import pickle
import pandas
import os

from geopy.geocoders import Nominatim

from multiprocessing.dummy import Pool

import numpy as np

from selenium import webdriver

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions

import time
import random

from datetime import datetime

geolocator = Nominatim(user_agent="PSP ADCRG Ohio")

In [2]:
batches = []

for i in range(290):
    print(i)
    
    pickle_in = open(os.getcwd()+'/data/pickle_dumps/batch'+str(i)+'.pickle', 'rb')
    batches.append(pickle.load(pickle_in))
    pickle_in.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [3]:
all_df = batches[0]

print(all_df.shape)

for i in batches[1:]:
    all_df = all_df.append(i)
    
all_df['latitude'] = 0.0
all_df['longitude'] = 0.0
all_df['last_transfer'] = datetime.strptime('1/1/1900', '%m/%d/%Y')

(100, 15)


In [4]:
def get_periods_over(a_df, delinquent_period):
    return a_df[a_df['periods_delinquent'] > delinquent_period]

def get_del_tax_over(a_df, amount):
    return a_df[a_df['delinquent_re_tax'] >= amount]

def not_classes(a_df, skip_classes):
    return a_df.loc[~a_df['property_class'].isin(skip_classes)]

def get_last_transfer_before(a_df, date):
    return a_df[a_df['last_transfer'] < datetime.strptime(date, '%m/%d/%Y')]

def get_last_transfer_after(a_df, date):
    return a_df[a_df['last_transfer'] > datetime.strptime(date, '%m/%d/%Y')]

def avoid_owners(a_df, skip_owners):
    return a_df.loc[~a_df['owner_name_1'].isin(skip_owners)]


In [5]:
def set_ff_preferences():
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference('browser.download.dir', (os.getcwd()+'/data/addresses'))
    profile.set_preference('browser.download.manager.showWhenStarting', False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', "application/csv, text/csv")
    return profile

def create_empty_ff_driver():
    profile = set_ff_preferences()
    driver = webdriver.Firefox(executable_path = '/Applications/geckodriver',firefox_profile=profile)
    return driver

def create_driver_opening_url(a_url):
    driver = create_empty_ff_driver()
    driver.get(a_url)
    return driver

def clean_from_text(text):
#     cleaned_text = text.replace(",","").strip()
    cleaned_text = text.strip()
    return cleaned_text

In [6]:
def initialize_driver():
    main_url = 'https://wedge1.hcauditor.org/'
    a_driver = create_driver_opening_url(main_url)
    return a_driver

def navigate_search_page(a_driver, parcel_id):
    parcel_id_rbutton = WebDriverWait(a_driver, 10).until(
    expected_conditions.presence_of_element_located(
        (By.XPATH, '//*[@id="search_radio_parcel_id"]')))

    parcel_id_rbutton.click()

    parcel_input = WebDriverWait(a_driver, 10).until(
    expected_conditions.presence_of_element_located(
        (By.XPATH, '//*[@id="parcel_number"]')))

    parcel_input.clear()
    parcel_input.send_keys(parcel_id)
    
    search_button = WebDriverWait(a_driver, 10).until(
    expected_conditions.presence_of_element_located(
        (By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[3]/form[3]/div[2]/button[1]')))

    search_button.click()

def navigate_parcel_page(a_driver):
    transfers_button = WebDriverWait(driver, 10).until(
    expected_conditions.presence_of_element_located(
        (By.XPATH, '/html/body/div/div[4]/div/div[2]/div[1]/a[4]')))

    transfers_button.click()
    
def navigate_to_data(a_driver, parcel_id):
        navigate_search_page(a_driver, parcel_id)
        navigate_parcel_page(a_driver)

def navigate_back_to_search(a_driver):
    try:
        a_driver.back()
        a_driver.back()
    except:
        a_driver.get('https://wedge1.hcauditor.org/')
        
def get_last_transfer(a_driver):
    last_transfer = WebDriverWait(a_driver, 10).until(
        expected_conditions.presence_of_element_located(
            (By.XPATH, '/html/body/div/div[3]/div[2]/table/tbody/tr[2]/td[4]'))
    ).text
    
    return datetime.strptime(last_transfer, '%m/%d/%Y')


In [7]:


print(all_df.shape)

# http://codes.ohio.gov/oac/5703-25-10
skip_classes = [100,300,400,500]
applicable_classes = not_classes(all_df, skip_classes)
print(applicable_classes.shape)

p_o_4 = get_periods_over(applicable_classes, 5)
print(p_o_4.shape)

d_t_o_7 = get_del_tax_over(p_o_4, 7000)
print(d_y_o_7.shape)

df = not_owners = ['STATE OF OHIO', 'WHAM PROPERTIES II LLC']
avoid_owners(d_t_o_7, not_owners)

(28820, 18)
(21040, 18)
(368, 18)
(145, 18)


In [8]:
# fails = []

# for i,row in delinquent_df.iterrows():
#     index = row.name
    
#     delinquent_df.at[index, 'address'] = delinquent_df.at[index, 'address']+' CINCINNATI'
    
#     try:
#         location = geolocator.geocode(delinquent_df.at[index, 'address'])
        
#         delinquent_df.at[index, 'latitude'] = location.latitude
#         delinquent_df.at[index, 'longitude'] = location.longitude
        
#     except:
#         fails.append(row.parcel_number)

In [9]:
df

,parcel_number,owner_name_1,owner_name_2,owner_address_1,owner_address_2,property_class,unpaid_amount,contract_yn,tax_district,address,delinquent_re_tax,gross_re_tax,periods_delinquent,land_value,building_value,latitude,longitude,last_transfer
240,0160001008600,BOWLING BILLY P,NaN,4509 EASTERN AVE,"CINCINNATI,OH 452261804",510,12311.06,N,1,4500 EASTERN AVE,8903.82,1622.16,5.489,20790.0,20790.0,0.0,0.0,1900-01-01
927,0360004014800,LOVDAL LISA M,NaN,1990 WOLFANGEL RD,"CINCINNATI,OH 45255",510,13993.05,N,1,4607 MATHIS ST,9622.82,1785.16,5.390,10910.0,10910.0,0.0,0.0,1900-01-01
1044,0370002002300,SUMLER KENNETH,NaN,4408 PLAINVILLE RD,"CINCINNATI,OH 452273261",510,19823.05,N,1,4408 PLAINVILLE RD,17096.72,2818.68,6.066,15270.0,15270.0,0.0,0.0,1900-01-01
1799,0540002012900,HAMILTON COUNTY LAND REUTILIZATION CORPORATION,NaN,3 EAST FOURTH ST SUITE 300,"CINCINNATI,OH 45202",680,297.90,N,1,1919 KINNEY AVE,16379.25,538.84,30.397,13600.0,13600.0,0.0,0.0,1900-01-01
1804,0540003000400,KAZ00 68 LLC,NaN,1894 EAST GALBRAITH AVE,"CINCINNATI,OH 45215",429,756.78,N,1,3250 WOLD AVE,17441.90,1203.32,14.495,9800.0,9800.0,0.0,0.0,1900-01-01
1849,0540004000600,CALHOUN JOHN A,NaN,1824 HURON AVE,"CINCINNATI,OH 452071604",510,17307.82,N,1,1824 HURON AVE,15772.88,1239.54,12.725,13600.0,13600.0,0.0,0.0,1900-01-01
1897,0540004013700,HAMILTON COUNTY LAND REUTILIZATION CORPORATION,NaN,3 EAST COURT ST SUITE 300,"CINCINNATI,OH 45202",680,64.78,N,1,1823 HEWITT AVE,12684.08,623.74,20.336,15740.0,15740.0,0.0,0.0,1900-01-01
2006,0550001007000,CASEY WILLIAM TROY,NaN,3131 HARVARD AVE,"CINCINNATI,OH 452071719",510,17325.35,N,1,3131 HARVARD AVE,13911.33,1238.40,11.233,7930.0,7930.0,0.0,0.0,1900-01-01
2022,0550002003200,WISDOM TERRENCE,NaN,1524 GILPIN AVE,"CINCINNATI,OH 45206",510,20275.80,N,1,1524 GILPIN AVE,18039.91,2181.36,8.270,9080.0,9080.0,0.0,0.0,1900-01-01
2033,0550002007600,BROOKS MARY E,NaN,3112 DURRELL AVE,"CINCINNATI,OH 452071718",520,18582.94,N,1,3112 DURRELL AVE,15866.97,2729.26,5.814,19140.0,19140.0,0.0,0.0,1900-01-01


In [10]:
driver = initialize_driver()

In [11]:
fails = []

for i,row in df.iterrows():
    index = row.name
    
    try:
        
        navigate_to_data(driver, row['parcel_number'])
        
        df.at[index, 'last_transfer'] = get_last_transfer(driver)
            
    except:
        fails.append(row['parcel_number'])
    
    navigate_back_to_search(driver)
    

In [12]:
navigate_back_to_search(driver)

In [13]:
df[df['last_transfer'] < datetime.strptime('1/1/2018', '%m/%d/%Y')]

,parcel_number,owner_name_1,owner_name_2,owner_address_1,owner_address_2,property_class,unpaid_amount,contract_yn,tax_district,address,delinquent_re_tax,gross_re_tax,periods_delinquent,land_value,building_value,latitude,longitude,last_transfer
1044,0370002002300,SUMLER KENNETH,NaN,4408 PLAINVILLE RD,"CINCINNATI,OH 452273261",510,19823.05,N,1,4408 PLAINVILLE RD,17096.72,2818.68,6.066,15270.0,15270.0,0.0,0.0,2012-07-25
1849,0540004000600,CALHOUN JOHN A,NaN,1824 HURON AVE,"CINCINNATI,OH 452071604",510,17307.82,N,1,1824 HURON AVE,15772.88,1239.54,12.725,13600.0,13600.0,0.0,0.0,2005-08-05
2006,0550001007000,CASEY WILLIAM TROY,NaN,3131 HARVARD AVE,"CINCINNATI,OH 452071719",510,17325.35,N,1,3131 HARVARD AVE,13911.33,1238.40,11.233,7930.0,7930.0,0.0,0.0,2005-01-21
2022,0550002003200,WISDOM TERRENCE,NaN,1524 GILPIN AVE,"CINCINNATI,OH 45206",510,20275.80,N,1,1524 GILPIN AVE,18039.91,2181.36,8.270,9080.0,9080.0,0.0,0.0,2009-05-13
2033,0550002007600,BROOKS MARY E,NaN,3112 DURRELL AVE,"CINCINNATI,OH 452071718",520,18582.94,N,1,3112 DURRELL AVE,15866.97,2729.26,5.814,19140.0,19140.0,0.0,0.0,1900-01-01
2074,0550004013100,KRS LLC,NaN,P O BOX 531725,"CINCINNATI,OH 45253",510,13349.39,Y,1,1701 KINNEY AVE,13376.27,1307.46,10.231,7340.0,7340.0,0.0,0.0,2007-06-20
2295,0580004001200,CUTHBERTSON ROBERT D &,DIANE M,1848 CLARION AVE,"CINCINNATI,OH 45202",510,28986.99,N,1,1848 CLARION AVE,26170.41,2499.46,10.470,15300.0,15300.0,0.0,0.0,1998-04-27
2376,0590006002500,STEPHENS ROBELAN,NaN,P O BOX 531725,"CINCINNATI,OH 45253",520,12302.26,Y,1,3553 NEWTON AVE,12177.27,1315.38,9.258,10550.0,10550.0,0.0,0.0,2007-06-14
2763,0660002014200,GLASSMEYER WALTER MICHAEL,& WALTER,7870 SAGAMORE DR,"CINCINNATI,OH 45236",455,10429.43,N,1,1223 LINCOLN AVE,8906.53,859.18,10.366,15540.0,15540.0,0.0,0.0,2010-01-07
2818,0670001004800,SEBASTIAN RICARDO,ROBERTO,922 ROGERS PL,"CINCINNATI,OH 452062527",510,14374.48,N,1,922 ROGERS PL,13017.15,2139.48,6.084,9180.0,9180.0,0.0,0.0,1995-09-29


In [14]:
get_last_transfer_before(df, '1/1/2017').sort_values('last_transfer', ascending=False)

,parcel_number,owner_name_1,owner_name_2,owner_address_1,owner_address_2,property_class,unpaid_amount,contract_yn,tax_district,address,delinquent_re_tax,gross_re_tax,periods_delinquent,land_value,building_value,latitude,longitude,last_transfer
3835,0940008038800,TUCKER JOSEPH G,NaN,3241 STANHOPE AVE,"CINCINNATI,OH 45211",404,14988.82,N,1,1635-1637 VINE ST,12478.33,2044.40,6.104,31630.0,31630.0,0.0,0.0,2016-08-05
12718,2060012003100,HOLLIE CHRIS,NaN,4019 GILMORE AVE,"CINCINNATI,OH 45209",510,17305.24,N,1,2928 WESTKNOLLS LN,14599.87,2913.76,5.011,11880.0,11880.0,0.0,0.0,2016-06-06
6143,1310006024800,KINDELL RICKY E & WILLA MAE KINDELL,NaN,2306 PARK AVE C/O JOHN J MULVEY,"CINCINNATI,OH 45206",510,24054.49,N,1,1321 CAROLINA AVE,16191.27,1850.82,8.748,13440.0,13440.0,0.0,0.0,2016-05-23
5339,1170011007800,CHYTIRIO PROPERTIES LLC,NaN,8580 KENTLAND CT,"CINCINNATI,OH 45236",447,22067.02,Y,1,1874 SECTION RD,26010.52,2695.30,9.650,50570.0,50570.0,0.0,0.0,2015-07-15
5888,1270001003200,HARPER SONIA L,& DOMINIQUE L HARPER,3747 WOODFORD RD,"CINCINNATI,OH 452132209",510,25703.05,N,1,3747 WOODFORD RD,21937.85,3710.70,5.912,44600.0,44600.0,0.0,0.0,2015-05-06
4717,1080001008700,WILLIAMS JOSEPH,NaN,3578 VINE ST,"CINCINNATI,OH 45220",401,18402.95,N,1,824 HUTCHINS AVE,15096.88,2541.34,5.941,14720.0,14720.0,0.0,0.0,2014-08-07
19534,5500142000800,SALETA LUIS O RIVA,NaN,8188 JORDAN VALLEY CT,"CLEVES,OH 45002",510,37453.72,N,9,1840 ANDERSON FERRY RD,8955.53,1347.18,6.648,33350.0,33350.0,0.0,0.0,2014-05-28
22399,5900350050000,BURTON JAMES E @ 20,NaN,2056 SECOND AVE,"CINCINNATI,OH 452241804",510,23778.52,N,134,2056 SECOND AVE,20238.54,3809.94,5.312,9380.0,9380.0,0.0,0.0,2013-11-26
3848,0950002000300,DONOHOO DEVELOPMENT LLC,NaN,4030 MT CARMEL TOBASCO RD #112,"CINCINNATI,OH 45255",350,46711.01,N,1,244 W MCMICKEN AVE,38671.86,6908.60,5.598,14880.0,14880.0,0.0,0.0,2012-12-05
3847,0950002000100,DONOHOO DEVELOPMENT LLC,NaN,4030 MT CARMEL TOBASCO RD #112,"CINCINNATI,OH 45255",350,94761.49,N,1,246 W MCMICKEN AVE,36663.37,2167.78,16.913,22730.0,22730.0,0.0,0.0,2012-12-05


In [15]:
fails

['0980005003900',
 '0990001007800',
 '1700009013000',
 '1710010001000',
 '2120064005090',
 '2160048003400',
 '5910004026400',
 '5910004045800']

In [16]:
df = get_last_transfer_before(df, '1/1/2017').sort_values('last_transfer', ascending=False)

In [17]:
# ignore wham properties
# if wham goes into a folder of review occasionally

In [18]:
# get_last_transfer_after(lt_df, '1/1/1901').sort_values('last_transfer', ascending=False)

In [20]:
not_owners = ['STATE OF OHIO', 'WHAM PROPERTIES II LLC']

fl_df = avoid_owners(df, not_owners)

In [22]:
pickle_out = open(os.getcwd()+'/data/pickle_dumps/finallist.pickle', 'wb')
pickle.dump(fl_df,pickle_out)
pickle_out.close()